# Adaptive Tool Routing (ATR) - Code Example

This notebook demonstrates **Adaptive Tool Routing** - a pattern for dynamically filtering tools based on user queries to reduce context bloat and improve agent performance.

## The Problem

When agents have access to many tools (e.g., 50+ MCP tools), two things happen:
1. **Context explosion** - Tool definitions consume 10,000-15,000 tokens before the conversation starts
2. **Tool selection degradation** - Research shows 7-85% accuracy drops with large tool catalogs

## The Solution

ATR intercepts the agent's tool resolution flow and filters tools *before* they reach the system prompt, using a lightweight LLM (gpt-4o-mini) to select only relevant tools based on the user's query.

In [1]:
# Install dependencies
%pip install agno openai yfinance python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "..."

In [3]:
from typing import List, Optional, Union

from agno.agent import Agent
from agno.models.base import Model
from agno.models.openai import OpenAIChat
from agno.run.agent import RunOutput
from agno.run.base import RunContext
from agno.session.agent import AgentSession
from agno.tools.function import Function
from agno.tools.toolkit import Toolkit
from agno.tools.yfinance import YFinanceTools
from agno.utils.log import log_debug, log_info

# Verify API key is set
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your .env file"
print("✓ OpenAI API key loaded")

✓ OpenAI API key loaded


## FilteredAgent Implementation

The `FilteredAgent` subclasses Agno's `Agent` and overrides `_determine_tools_for_model()` to inject the filtering step.

**Key components:**
1. **Filter Agent** - A lightweight gpt-4o-mini agent that selects relevant tools
2. **Tool Prompt Builder** - Creates minimal tool summaries (name + truncated description)
3. **Override Hook** - Intercepts tool resolution before tools reach the system prompt

In [4]:
class FilteredAgent(Agent):
    """
    Agent that filters tools based on user input using a small LLM.
    
    This reduces context size and prevents the main model from getting confused
    when many tools are available but only a few are relevant to the query.
    """

    def __init__(
        self,
        filter_model: str = "gpt-4o-mini",
        filter_enabled: bool = True,
        max_tools: int = 5,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.filter_model_id = filter_model
        self.filter_enabled = filter_enabled
        self.max_tools = max_tools
        self._filter_agent: Optional[Agent] = None
        self._current_input_text: Optional[str] = None

    def _get_filter_agent(self) -> Agent:
        """Lazily create the tool filter agent."""
        if self._filter_agent is None:
            self._filter_agent = Agent(
                model=OpenAIChat(id=self.filter_model_id),
                instructions=[
                    "You are a tool selector assistant.",
                    "Given a user query and a list of available tools, your job is to select ONLY the tools that are directly relevant to answering the query.",
                    "Be conservative - select only tools that will definitely be needed.",
                    "Return ONLY the tool names, one per line, no explanations or numbering.",
                    "If unsure between similar tools, include both.",
                ],
                markdown=False,
            )
        return self._filter_agent

    def _extract_input_text(self, run_response: RunOutput) -> Optional[str]:
        """Extract the user input text from run_response."""
        if run_response.input is None:
            return None
        try:
            return run_response.input.input_content_string()
        except Exception:
            input_obj = run_response.input
            if hasattr(input_obj, "input_content"):
                content = input_obj.input_content
                if isinstance(content, str):
                    return content
            return None

    def _build_tools_prompt(self, input_text: str, functions: List[Function]) -> str:
        """Build the prompt for the tool filter agent."""
        tool_descriptions = []
        for f in functions:
            if isinstance(f, Function):
                desc = f.description or "No description"
                # Truncate long descriptions to save tokens
                if len(desc) > 150:
                    desc = desc[:147] + "..."
                tool_descriptions.append(f"- {f.name}: {desc}")

        tools_list = "\n".join(tool_descriptions)

        return f"""User query: "{input_text}"

Available tools:
{tools_list}

Select the tools needed for this query. Return only tool names, one per line."""

    def _filter_tools_sync(
        self, input_text: str, functions: List[Union[Function, dict]]
    ) -> List[Union[Function, dict]]:
        """Filter tools using the filter agent (synchronous)."""
        # Separate Functions from dicts (builtin tools)
        function_objs = [f for f in functions if isinstance(f, Function)]
        dict_tools = [f for f in functions if isinstance(f, dict)]

        if not function_objs:
            return functions

        prompt = self._build_tools_prompt(input_text, function_objs)

        try:
            filter_agent = self._get_filter_agent()
            response = filter_agent.run(prompt)

            if response.content and not response.content.startswith("Error"):
                # Parse selected tool names
                selected_names = set(
                    line.strip().lstrip("- ").strip()
                    for line in response.content.strip().split("\n")
                    if line.strip() and not line.strip().startswith("Error")
                )

                all_tool_names = {f.name for f in function_objs}
                valid_selected = selected_names & all_tool_names

                if valid_selected:
                    log_info(f"Tool filter selected: {valid_selected}")
                    filtered = [f for f in function_objs if f.name in valid_selected]

                    # Apply max_tools limit
                    if len(filtered) > self.max_tools:
                        filtered = filtered[: self.max_tools]

                    return filtered + dict_tools
                else:
                    log_debug("No valid tools selected, using all tools")
                    return functions

        except Exception as e:
            log_debug(f"Tool filtering failed: {e}, using all tools")
            return functions

        return functions

    def _determine_tools_for_model(
        self,
        model: Model,
        processed_tools: List[Union[Toolkit, callable, Function, dict]],
        run_response: RunOutput,
        run_context: RunContext,
        session: AgentSession,
    ) -> List[Union[Function, dict]]:
        """Override to filter tools before they're sent to the model."""
        # Get all functions from parent (flattens Toolkits into Functions)
        all_functions = super()._determine_tools_for_model(
            model, processed_tools, run_response, run_context, session
        )

        if not self.filter_enabled or not all_functions:
            return all_functions

        input_text = self._extract_input_text(run_response)
        if not input_text:
            log_debug("Could not extract input text, using all tools")
            return all_functions

        log_info(f"Filtering {len(all_functions)} tools for query: '{input_text[:50]}...'")
        filtered = self._filter_tools_sync(input_text, all_functions)
        log_info(f"Filtered to {len(filtered)} tools")

        return filtered

print("✓ FilteredAgent class defined")

✓ FilteredAgent class defined


## Demo: YFinance Tools

YFinance toolkit provides **9 tools**:
- `get_current_stock_price` - Current price for a ticker
- `get_company_info` - Company information and description
- `get_stock_fundamentals` - Fundamental data (market cap, P/E, etc.)
- `get_income_statements` - Income statement data
- `get_key_financial_ratios` - Financial ratios
- `get_analyst_recommendations` - Analyst buy/sell recommendations
- `get_company_news` - Recent news articles
- `get_technical_indicators` - RSI, MACD, Bollinger Bands
- `get_historical_stock_prices` - Historical price data

Without filtering, all 9 tools (~2,000+ tokens) go into the system prompt for every query.
With ATR, only the relevant 1-3 tools are included.

In [ ]:
# Create the FilteredAgent with YFinance tools
agent = FilteredAgent(
    model=OpenAIChat(id="gpt-4o"),      # Main model for responses
    filter_model="gpt-4o-mini",          # Small/fast model for tool filtering
    filter_enabled=True,                 # Enable ATR filtering
    max_tools=5,                         # Cap filtered tools at 5
    tools=[YFinanceTools()],             # All 9 YFinance tools
    markdown=True,
    debug_mode=True,                     # Shows tool calls in output
)

print("✓ FilteredAgent created with YFinance tools (9 tools available)")

✓ FilteredAgent created with YFinance tools (9 tools available)


## Test Query 1: Simple Price Lookup

**Query:** "What is the current price of AAPL?"

**Expected filtered tools:** `get_current_stock_price` only

In [12]:
response = agent.run("What is the current price of AAPL?")
print(response.content)

DEBUG ***** Session ID: 6d957cd4-35d0-4333-afa6-ca450325dc8b *****

DEBUG ***** Session ID: 6d957cd4-35d0-4333-afa6-ca450325dc8b *****

DEBUG ****** Agent ID: f0d796eb-b979-4c10-a018-3df1de1f226a ******

DEBUG Creating new AgentSession: 6d957cd4-35d0-4333-afa6-ca450325dc8b

DEBUG Processing tools for model

...

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

INFO Tool filter selected: {'get_current_stock_price'}

INFO Filtered to 1 tools

The current price of AAPL (Apple Inc.) stock is $248.04.


## Test Query 2: Multi-Tool Query

**Query:** "What are analysts saying about NVDA and any recent news?"

**Expected filtered tools:** `get_analyst_recommendations`, `get_company_news`

This demonstrates ATR selecting multiple relevant tools for a complex query.

In [8]:
response = agent.run("What are analysts saying about NVDA and any recent news?")
print(response.content)

DEBUG ****** Agent ID: f0d796eb-b979-4c10-a018-3df1de1f226a ******

DEBUG Creating new AgentSession: 6d957cd4-35d0-4333-afa6-ca450325dc8b

DEBUG Processing tools for model

DEBUG Added tool get_current_stock_price from yfinance_tools

DEBUG Added tool get_company_info from yfinance_tools

...

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

INFO Tool filter selected: {'get_analyst_recommendations', 'get_company_news'}

INFO Filtered to 2 tools

### Analyst Recommendations for NVIDIA (NVDA)

**Current Month:**
- **Strong Buy:** 12
- **Buy:** 48
- **Hold:** 3
- **Sell:** 1
- **Strong Sell:** 0

**Previous Months:**
- **-1 Month: Strong Buy:** 11, **Buy:** 49, **Hold:** 3, **Sell:** 1, **Strong Sell:** 0
- **-2 Months: Strong Buy:** 11, **Buy:** 49, **Hold:** 3, **Sell:** 1, **Strong Sell:** 0
- **-3 Months: Strong Buy:** 10, **Buy:** 48, **Hold:** 4, **Sell:** 1, **Strong Sell:** 0

The analyst sentiment remains overwhelmingly positive towards NVIDIA, with a consistent majority in the "Strong Buy" and "Buy" categories over the past months.

### Recent News for NVIDIA (NVDA)

1. [**AI power and infrastructure needs boomed in 2025. At Davos, the AI story for 2026 remains the same.**](https://finance.yahoo.com/news/ai-power-and-infrastructure-needs-boomed-in-2025-at-davos-the-ai-story-for-2026-remains-the-same-100005093.html)
   - **Summary:** Discussions at Davos emphasized that the need for AI infrastructure is expected to conti

## Test Query 3: Technical Analysis

**Query:** "Show me Tesla's historical prices and technical indicators"

**Expected filtered tools:** `get_historical_stock_prices`, `get_technical_indicators`

In [9]:
response = agent.run("Show me Tesla's historical prices and technical indicators")
print(response.content)

DEBUG ****** Agent ID: f0d796eb-b979-4c10-a018-3df1de1f226a ******

DEBUG Creating new AgentSession: 6d957cd4-35d0-4333-afa6-ca450325dc8b

DEBUG Processing tools for model

DEBUG Added tool get_current_stock_price from yfinance_tools

DEBUG Added tool get_company_info from yfinance_tools

...

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

INFO Tool filter selected: {'get_technical_indicators', 'get_historical_stock_prices'}

INFO Filtered to 2 tools

Here are Tesla's historical prices and technical indicators for the past month:

### Historical Prices (Last Month)
Here is a summary of Tesla's daily historical stock prices over the last month. The data includes the open, high, low, and close prices, along with the volume of shares traded on each day.

| Date           | Open   | High   | Low    | Close  | Volume   |
|----------------|--------|--------|--------|--------|----------|
| Nov 1, 2023    | 488.48 | 490.90 | 476.80 | 485.40 | 41,285,400 |
| Nov 2, 2023    | 485.23 | 489.09 | 473.82 | 475.19 | 58,780,700 |
| Nov 5, 2023    | 469.00 | 469.40 | 459.00 | 459.64 | 66,263,000 |
| Nov 6, 2023    | 461.09 | 463.12 | 453.83 | 454.43 | 59,238,500 |
| Nov 7, 2023    | 456.10 | 456.55 | 449.30 | 449.72 | 49,078,000 |
| Nov 9, 2023    | 457.80 | 458.34 | 435.30 | 438.07 | 85,535,400 |
| Nov 12, 2023   | 447.99 | 457.55 | 444.57 | 451.67 | 67,940,800 |
| Nov 13, 2023   | 446.38 | 448.25 | 428.78 | 432.96 | 89,093,800 |
| Nov 14, 2023   |

## Test Query 4: Fundamentals

**Query:** "What's Google's P/E ratio and other fundamentals?"

**Expected filtered tools:** `get_stock_fundamentals`, `get_key_financial_ratios`

In [10]:
response = agent.run("What's Google's P/E ratio and other fundamentals?")
print(response.content)

DEBUG ****** Agent ID: f0d796eb-b979-4c10-a018-3df1de1f226a ******

DEBUG Creating new AgentSession: 6d957cd4-35d0-4333-afa6-ca450325dc8b

DEBUG Processing tools for model

DEBUG Added tool get_current_stock_price from yfinance_tools

DEBUG Added tool get_company_info from yfinance_tools

...

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

DEBUG ***** Session ID: 93caeef6-7340-4df8-858a-c7dc9f4fec83 *****

INFO Tool filter selected: {'get_key_financial_ratios', 'get_stock_fundamentals'}

INFO Filtered to 2 tools

Here are the latest fundamental data and key financial ratios for Alphabet Inc. (GOOGL):

### Company Overview
- **Name:** Alphabet Inc.
- **Sector:** Communication Services
- **Industry:** Internet Content & Information
- **Market Capitalization:** $3.97 trillion
- **Headquarters:** Mountain View, California, United States
- **CEO:** Mr. Sundar Pichai
- **Website:** [abc.xyz](https://abc.xyz)

### Stock Fundamentals
- **P/E Ratio:** 29.17
- **P/B Ratio:** 10.24
- **Dividend Yield:** 0.26%
- **Earnings Per Share (EPS):** $10.13
- **Beta:** 1.086
- **52-Week High:** $340.49
- **52-Week Low:** $140.53

### Key Financial Ratios
- **Price to Book Ratio:** 10.24
- **Price to Sales Ratio (TTM):** 10.30
- **Trailing P/E:** 32.37
- **Forward P/E:** 29.17
- **Enterprise Value to Revenue:** 10.13
- **Enterprise to EBITDA:** 26.88
- **Debt to Equity:** 11.42%
- **Current Ratio:** 1.75
- **Quick Ratio:** 1.56

### Additional Financials
- **Total Revenue:** $385.48 billion
- **Net Income:** $124.25

## Comparison: With vs Without Filtering

Let's compare behavior with filtering disabled to see the difference.

In [13]:
# Create an agent WITHOUT filtering for comparison
agent_no_filter = FilteredAgent(
    model=OpenAIChat(id="gpt-4o"),
    filter_enabled=False,  # Filtering DISABLED
    tools=[YFinanceTools()],
    markdown=True,
    debug_mode=True,
)

print("Agent without filtering - all 9 tools sent to model on every query")
print("-" * 60)
response = agent_no_filter.run("What is the current price of AAPL?")
print(response.content)

Agent without filtering - all 9 tools sent to model on every query
------------------------------------------------------------


DEBUG ****** Agent ID: 176f1b7e-ab13-481c-8f53-5d9a9425dd38 ******

DEBUG Creating new AgentSession: edc385c1-5c8e-4575-8543-76d45b5e8f6a

DEBUG Processing tools for model

DEBUG Added tool get_current_stock_price from yfinance_tools

...

DEBUG ************************  METRICS  *************************

DEBUG ------------------- OpenAI Response End --------------------

DEBUG Added RunOutput to Agent Session

DEBUG *** Agent Run End: 47597fd6-9c0c-4b92-80e2-06350319412e ****

The current price of AAPL (Apple Inc.) stock is $248.04.


## Summary

**What ATR does:**
1. Intercepts tool resolution in `_determine_tools_for_model()`
2. Passes user query + tool summaries to a lightweight filter agent (gpt-4o-mini)
3. Filter agent returns only relevant tool names
4. Filtered tools are returned instead of full list

**Benefits:**
- **90% context reduction** (50 tools × 250 tokens → 5 tools × 250 tokens)
- **Improved tool selection accuracy** - fewer options means less confusion
- **Lower costs** - less input tokens per request
- **Minimal overhead** - ~100ms, ~200-400 tokens for the routing call

**When to use:**
- 15+ tools configured
- Diverse tool domains
- Cost/latency sensitive applications

See the full blog post for advanced patterns like feedback loops and RAG-based tool discovery for 500+ tool scenarios.